In [ ]:
# !pip install smdebug -q
# !pip install matplotlib -q

In [1]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from tensorflow.python.framework.ops import disable_eager_execution

import time
import matplotlib.pyplot as plt

In [2]:
disable_eager_execution()
tf.executing_eagerly()

False

In [3]:
(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()
train_images, test_images = train_images / 255.0, test_images / 255.0

class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer',
               'dog', 'frog', 'horse', 'ship', 'truck']

In [4]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10))

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [5]:
import smdebug.tensorflow as smd

job_name = 'tf-exp' + time.strftime("%m-%d-%H-%M-%S", time.gmtime())
hook = smd.KerasHook(out_dir=f'./smd_outputs/{job_name}', 
                     tensorboard_dir=f'./tb_logs/{job_name}',
                     save_config=smd.SaveConfig(save_interval=1),
                     include_collections=['gradients', 'biases'])

opt = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.9, name='SGD')
opt = hook.wrap_optimizer(opt)

model.compile(optimizer = opt,
              loss      = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics   = ['accuracy'])

[2020-03-30 06:05:05.940 ip-172-31-28-67:31267 INFO hook.py:215] Saving to ./smd_outputs/tf-exp03-30-06-05-05


In [6]:
history = model.fit(train_images, train_labels, 
                    epochs=2, batch_size=256,
                    validation_data=(test_images, test_labels),
                    callbacks=[hook])

Train on 50000 samples, validate on 10000 samples
Epoch 1/2
[2020-03-30 06:06:07.670 ip-172-31-28-67:31267 INFO hook.py:351] Monitoring the collections: metrics, biases, losses, gradients, sm_metrics
50000/50000 [==============================] - 29s 583us/sample - loss: 2.0541 - accuracy: 0.2384 - val_loss: 1.7498 - val_accuracy: 0.3656
Epoch 2/2
50000/50000 [==============================] - 27s 547us/sample - loss: 1.6338 - accuracy: 0.4082 - val_loss: 1.5354 - val_accuracy: 0.4373


In [7]:
# Create a trial to inspect the saved tensors
trial = smd.create_trial(path=f'./smd_outputs/{job_name}')

[2020-03-30 06:07:04.164 ip-172-31-28-67:31267 INFO local_trial.py:35] Loading trial tf-exp03-30-06-05-05 at path ./smd_outputs/tf-exp03-30-06-05-05


In [8]:
trial.tensor_names()

['accuracy',
 'batch',
 'conv2d/weights/conv2d/bias:0',
 'conv2d_1/weights/conv2d_1/bias:0',
 'conv2d_2/weights/conv2d_2/bias:0',
 'dense/weights/dense/bias:0',
 'dense_1/weights/dense_1/bias:0',
 'loss',
 'size',
 'training/SGD/gradients/gradients/conv2d/BiasAdd_grad/BiasAddGrad:0',
 'training/SGD/gradients/gradients/conv2d/Conv2D_grad/Conv2DBackpropFilter:0',
 'training/SGD/gradients/gradients/conv2d_1/BiasAdd_grad/BiasAddGrad:0',
 'training/SGD/gradients/gradients/conv2d_1/Conv2D_grad/Conv2DBackpropFilter:0',
 'training/SGD/gradients/gradients/conv2d_2/BiasAdd_grad/BiasAddGrad:0',
 'training/SGD/gradients/gradients/conv2d_2/Conv2D_grad/Conv2DBackpropFilter:0',
 'training/SGD/gradients/gradients/dense/BiasAdd_grad/BiasAddGrad:0',
 'training/SGD/gradients/gradients/dense/MatMul_grad/MatMul_1:0',
 'training/SGD/gradients/gradients/dense_1/BiasAdd_grad/BiasAddGrad:0',
 'training/SGD/gradients/gradients/dense_1/MatMul_grad/MatMul_1:0',
 'val_accuracy',
 'val_loss']

In [9]:
trial.tensor('conv2d/weights/conv2d/bias:0').values()

{0: array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       dtype=float32),
 1: array([-1.53493078e-04, -9.57493830e-05,  2.63235188e-06, -2.37621080e-05,
         2.99976145e-05,  0.00000000e+00,  7.68151324e-07, -1.38967735e-05,
        -1.24669197e-04, -6.14109813e-05, -4.01330362e-05, -1.85209690e-04,
        -8.87167625e-05,  1.15500516e-04,  1.36345516e-05,  1.11076167e-06,
         2.34559229e-06, -3.16375304e-06, -1.04484034e-05, -4.87506186e-05,
        -2.45362844e-05,  8.34997554e-05,  8.17475229e-05, -1.57642990e-06,
         1.47507524e-06, -4.58334080e-06, -3.69643640e-05,  7.08796006e-06,
        -1.42426754e-04, -1.12236849e-04, -2.26924269e-04,  2.06152326e-05],
       dtype=float32),
 2: array([-3.3424300e-04, -1.9768759e-04, -9.8680885e-06, -2.8831066e-05,
         5.4695112e-05, -4.7372305e-07, -7.7347831e-06, -3.9908067e-05,
        -1.5866981e-04, -7.6921810e-05, -2.2343